In [2]:
import os
import numpy as np
import h5py

In [3]:
file_11 = []
file_12 = []
file_29 = []
file_30 = []
file_39 = []

def walk(path):
    if not os.path.exists(path):
        print("path error")
        return -1
    for root, dirs, filenames in os.walk(path):
        for filename in filenames:
            if str(filename) == '000011.SZtick.mat':
                file_11.append(os.path.join(root,filename))
            elif str(filename) == '000012.SZtick.mat':
                file_12.append(os.path.join(root,filename))
            elif str(filename) == '000029.SZtick.mat':
                file_29.append(os.path.join(root,filename))
            elif str(filename) == '000030.SZtick.mat':
                file_30.append(os.path.join(root,filename))
            elif str(filename) == '000039.SZtick.mat':
                file_39.append(os.path.join(root,filename))
            else:
                print("unknown file name")
                return -1
    return 0

In [6]:
ret = walk(r'data-15.07.08')
if ret == -1:
    assert False

In [36]:
def calc_b(SZtick_filenames):
    b_all = 0
    for dataPath in SZtick_filenames:
        f = h5py.File(dataPath,'r+')
        f['r1'].keys()
        keys = ['AccTurover', 'AccVolume', 'AskAvPrice', 'AskPrice', 'AskVolume', 'BSFlag', 'BidAvPrice', 'BidPrice', 'BidVolume', 'Code', 'CurDelta', 'Date', 'Downs', 'High', 'HoldLines', 'Index', 'Interest', 'Low', 'MatchItems', 'Open', 'Position', 'PreClose', 'PrePosition', 'PreSettle', 'Price', 'Settle', 'Stocks', 'Time', 'TotalAskVolume', 'TotalBidVolume', 'TradeFlag', 'Turover', 'Ups', 'Volume', 'WindCode']
        data = []
        for key in keys:
            data.append(f['r1'][key])
        BidPrice = np.array(f['r1']['BidPrice'])
        AskPrice = np.array(f['r1']['AskPrice'])
        BidVolume = np.array(f['r1']['BidVolume'])
        AskVolume = np.array(f['r1']['AskVolume'])
        # TotalBidVolume = np.array(f['r1']['TotalBidVolume'])## 添加于2.12.16.55
        # TotalAskVolume = np.array(f['r1']['TotalAskVolume'])
        TotalBidVolume = np.array(np.sum(BidPrice, axis=0),dtype=int)
        TotalAskVolume = np.array(np.sum(AskPrice, axis=0),dtype=int)
        Price = np.array(f['r1']['Price']) 
        miu_n = BidVolume.astype(int) - AskVolume.astype(int)
        Total_miu_n = TotalBidVolume.astype(int) - TotalAskVolume.astype(int) 
        Total_miu_n = Total_miu_n.reshape(1, -1)

        Price_change = np.zeros(Price.shape)
        Total_miu_n_change = np.zeros(Total_miu_n.shape)
        # Price_change[0,1:Y_Price-1] = Price[0,1:Y_Price-1] - Price[0,0:Y_Price-2]

        [dummy,Y_totmiun] = Total_miu_n.shape
        [dummy,Y_Price] = Price.shape

        DIST_TOTALMIU = 1
        for i in range(DIST_TOTALMIU,Y_totmiun): 
            Total_miu_n_change[0,i] = Total_miu_n[0,i] - Total_miu_n[0,i-DIST_TOTALMIU]
        for i in range(0,DIST_TOTALMIU):
            Total_miu_n_change[0,i] = Total_miu_n[0,i]

        '''
        DIST_PRICE = 5
        for i in range(DIST_PRICE,Y_Price-1): 
            Price_change[0,i] = Price[0,i] - Price[0,i-DIST_PRICE]
            if Price_change[0,i] > 1:  #强行删去一个很偏离的点
                Price_change[0,i] = 0
        for i in range(1,DIST_PRICE-1):
            Price_change[0,i] = Price[0,i]
        '''

        DIST_PRICE = 5
        Price_change = np.zeros(Price.shape)
        # Price_change[0,1:Y_Price-1] = Price[0,1:Y_Price-1] - Price[0,0:Y_Price-2]
        for i in range(DIST_PRICE,Y_Price): 
            Price_change[0,i] = Price[0,i] - Price[0,i-DIST_PRICE]
            if Price_change[0,i] > 1:  #强行删去一个很偏离的点
                Price_change[0,i] = 0
        for i in range(0, DIST_PRICE):
            Price_change[0,i] = Price[0,i]


        from sklearn.model_selection import train_test_split

        # temp1 =  np.arange(1,Y_totmiun).reshape(1,Y_totmiun-1)#转为1*Y_totmiun-1的矩阵，以使得shape和Total_miu_n匹配
        # temp2 =  np.arange(1,Y_totmiun).reshape(1,Y_totmiun-1)#但这种方式不被train_test_split接受，它要求两个都是数组而不是矩阵
        # X_train,X_test,Y_train,Y_test = train_test_split(temp1[0,0:Y_totmiun-1],temp2[0,0:Y_totmiun-1])

        # X_train,X_test,Y_train,Y_test = train_test_split(Total_miu_n_change[0,0:Y_totmiun-1],Price_change[0,0:Y_Price-1],test_size=0)

        from sklearn.linear_model import LinearRegression

        X_train = Total_miu_n_change[0,0:Y_totmiun]
        Y_train = Price_change[0,0:Y_Price]
        model=LinearRegression()
        model.fit(X_train.reshape(-1, 1),Y_train.reshape(-1, 1))
        # model.score(X_test.reshape(-1, 1),Y_test.reshape(-1, 1))


        a=model.intercept_#截距
        b=model.coef_#回归系数

        # print(b)

        b_all += b

    b_average = b_all / len(SZtick_filenames)
    return b_average

In [37]:
print(calc_b(file_11))
print(calc_b(file_12))
print(calc_b(file_29))
print(calc_b(file_30))
print(calc_b(file_39))

[[1.61144555e-07]]
[[4.23827577e-08]]
[[2.27322663e-07]]
[[2.49116032e-07]]
[[1.72804347e-07]]
